# APPLICATION TO HESTON MODEL

In [4]:
# Parameters for the Heston model
kappa = 3.0
theta = 0.04
sigma_v = 0.1
r = 0.05
rho = -0.1 # covariance between price and volatility brownian motions

## Analytical

In [5]:
# Discretisation
dw = 0.02

## Recombining Tree

In [6]:
# Compute the Markov chain

# upper and lower bounds for the discretization of the volatility
ml, mu=  15, 40
m = mu-ml+1
regimes = range(1,m+1)

def transition(k):

    forward = sigma_v**2/(2*(dw)**2) + (2*kappa*theta-sigma_v**2)/(2*k*(dw)**2) - kappa*k/4

    backward = sigma_v**2/(2*(dw)**2) - (2*kappa*theta-sigma_v**2)/(2*k*(dw)**2) + kappa*k/4

    return forward,backward


def compute_Q():
    
    Q = np.zeros((m,m))

    for i in range(1,m-1):
        
        forward,backward = transition(i+ml-1)

        if forward>=0 and backward>=0:

            Q[i,i+1] = forward
            Q[i,i-1] = backward
            Q[i,i] = -forward-backward

        elif forward>0:

            Q[i,i-1] = backward + kappa*(i+ml-1)/4
            Q[i,i+1] = sigma_v**2/(2*(dw)**2) 
            Q[i,i] = -forward-backward

        else:
            
            Q[i,i-1] = sigma_v**2/(2*(dw)**2) 
            Q[i,i+1] = forward - kappa*(i+ml-1)/4
            Q[i,i] = -forward-backward
    # i =0
    Q[i,i+1] = (2*kappa*theta-sigma_v**2)/(2*ml*(dw)**2) - kappa*ml/2
    Q[i,i] = - Q[i,i+1]
    # i = m-1
    Q[i,i-1] = -(2*kappa*theta-sigma_v**2)/(2*mu*(dw)**2) + kappa*mu/2
    Q[i,i] = - Q[i,i-1]

    return Q